In [ ]:
"""
Krishi-Patra: Crop Cycle Linked Dynamic Finance Platform
A prototype implementation for Google Colab with Gradio UI

This demo includes:
1. Plot Assessment using simulated satellite data
2. Crop Health Monitoring
3. Dynamic Risk Scoring
4. Loan Recommendation Engine
"""

# Install required packages
# Run this cell first in Colab:
# !pip install gradio numpy pandas scikit-learn xgboost matplotlib seaborn pillow

import gradio as gr
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from io import BytesIO
import base64

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

class CropHealthModel:
    """Simulates AI model for crop health assessment using NDVI and environmental data"""

    def __init__(self):
        self.scaler = StandardScaler()
        self.model = RandomForestRegressor(n_estimators=100, random_state=42)
        self._train_baseline()

    def _train_baseline(self):
        """Train on synthetic baseline data"""
        np.random.seed(42)
        n_samples = 1000

        # Features: NDVI, Soil Moisture, Temperature, Rainfall, Days Since Sowing
        X = np.random.rand(n_samples, 5)
        X[:, 0] = X[:, 0] * 0.9 + 0.1  # NDVI: 0.1-1.0
        X[:, 1] = X[:, 1] * 60 + 20     # Soil Moisture: 20-80%
        X[:, 2] = X[:, 2] * 20 + 15     # Temp: 15-35°C
        X[:, 3] = X[:, 3] * 200         # Rainfall: 0-200mm
        X[:, 4] = X[:, 4] * 120         # Days: 0-120

        # Health score based on conditions
        y = (X[:, 0] * 40 +                    # NDVI impact
             (X[:, 1] - 30).clip(0, 30) +      # Optimal soil moisture
             (30 - abs(X[:, 2] - 25)) +        # Optimal temp around 25°C
             (X[:, 3] / 5).clip(0, 20) +       # Rainfall impact
             np.random.randn(n_samples) * 5)    # Random variation
        y = np.clip(y, 0, 100)

        self.scaler.fit(X)
        X_scaled = self.scaler.transform(X)
        self.model.fit(X_scaled, y)

    def predict_health(self, ndvi, soil_moisture, temp, rainfall, days_since_sowing):
        """Predict crop health score (0-100)"""
        X = np.array([[ndvi, soil_moisture, temp, rainfall, days_since_sowing]])
        X_scaled = self.scaler.transform(X)
        health_score = self.model.predict(X_scaled)[0]
        return np.clip(health_score, 0, 100)

class DynamicRiskEngine:
    """Dynamic risk assessment and loan recommendation engine"""

    def __init__(self):
        self.base_interest_rate = 9.0  # Base rate in %

    def calculate_crop_viability_score(self, health_score, plot_potential,
                                       weather_risk, market_price_trend):
        """
        Calculate Crop Viability Score (0-100)
        This is the core innovation - scoring the crop, not just the farmer
        """
        weights = {
            'health': 0.40,
            'potential': 0.25,
            'weather': 0.20,
            'market': 0.15
        }

        viability = (health_score * weights['health'] +
                    plot_potential * weights['potential'] +
                    (100 - weather_risk) * weights['weather'] +
                    market_price_trend * weights['market'])

        return round(viability, 2)

    def assess_risk_category(self, viability_score):
        """Categorize risk based on viability score"""
        if viability_score >= 75:
            return "Low Risk", "green"
        elif viability_score >= 50:
            return "Medium Risk", "yellow"
        else:
            return "High Risk", "red"

    def calculate_dynamic_terms(self, viability_score, requested_amount):
        """Calculate dynamic loan terms based on crop viability"""
        risk_category, _ = self.assess_risk_category(viability_score)

        # Dynamic interest rate (7-12% range)
        if viability_score >= 75:
            interest_rate = 7.5
            max_loan = requested_amount
        elif viability_score >= 50:
            interest_rate = 9.0
            max_loan = requested_amount * 0.85
        else:
            interest_rate = 11.0
            max_loan = requested_amount * 0.60

        # Calculate repayment schedule
        tenure_months = 6  # Typical crop cycle
        monthly_rate = interest_rate / (12 * 100)
        emi = max_loan * monthly_rate * (1 + monthly_rate)**tenure_months / \
              ((1 + monthly_rate)**tenure_months - 1)

        return {
            'approved_amount': round(max_loan, 2),
            'interest_rate': interest_rate,
            'tenure_months': tenure_months,
            'emi': round(emi, 2),
            'total_repayment': round(emi * tenure_months, 2),
            'risk_category': risk_category
        }

    def recommend_interventions(self, health_score, weather_risk):
        """Proactive intervention recommendations"""
        recommendations = []

        if health_score < 60:
            recommendations.append({
                'type': 'Alert',
                'message': 'Crop health declining. Top-up loan for pesticides available.',
                'action': 'Apply for ₹5,000 emergency top-up'
            })

        if weather_risk > 70:
            recommendations.append({
                'type': 'Insurance',
                'message': 'High weather risk detected. Activate micro-insurance.',
                'action': 'Pay ₹500 premium for ₹20,000 coverage'
            })

        if health_score > 75 and weather_risk < 30:
            recommendations.append({
                'type': 'Opportunity',
                'message': 'Excellent crop conditions. Consider expansion loan.',
                'action': 'Eligible for additional ₹50,000 at 7.5% interest'
            })

        return recommendations

class PlotAssessment:
    """Assess plot potential using historical data simulation"""

    @staticmethod
    def calculate_plot_potential(district, soil_type, irrigation, prev_yield):
        """Calculate plot potential score (0-100)"""
        # Base scores
        soil_scores = {
            'Alluvial': 85,
            'Black': 80,
            'Red': 70,
            'Laterite': 60,
            'Sandy': 50
        }

        irrigation_scores = {
            'Canal': 90,
            'Borewell': 85,
            'Rainfed': 60
        }

        base_score = soil_scores.get(soil_type, 65)
        irrigation_bonus = irrigation_scores.get(irrigation, 70) * 0.3
        yield_factor = min(prev_yield / 30, 1) * 20  # Normalize to 30 quintals/hectare

        potential = base_score * 0.5 + irrigation_bonus + yield_factor
        return round(np.clip(potential, 0, 100), 2)

# Initialize models
crop_health_model = CropHealthModel()
risk_engine = DynamicRiskEngine()
plot_assessor = PlotAssessment()

def generate_health_trend_chart(health_score):
    """Generate a trend chart showing crop health over time"""
    days = np.arange(0, 90, 10)
    # Simulate historical trend leading to current score
    trend = health_score + np.random.randn(len(days)) * 8
    trend = np.clip(trend, 0, 100)
    trend[-1] = health_score  # Current score

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(days, trend, marker='o', linewidth=2, markersize=8, color='#2E7D32')
    ax.axhline(y=75, color='g', linestyle='--', alpha=0.3, label='Healthy Threshold')
    ax.axhline(y=50, color='orange', linestyle='--', alpha=0.3, label='Warning Threshold')
    ax.axhline(y=25, color='r', linestyle='--', alpha=0.3, label='Critical Threshold')

    ax.set_xlabel('Days Since Sowing', fontsize=12)
    ax.set_ylabel('Crop Health Score', fontsize=12)
    ax.set_title('Crop Health Trend Analysis', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0, 105)

    return fig

def assess_loan_application(district, soil_type, irrigation, prev_yield,
                           ndvi, soil_moisture, temp, rainfall, days_since_sowing,
                           weather_risk, market_trend, requested_amount):
    """Main function to assess loan application"""

    # Step 1: Plot Assessment
    plot_potential = plot_assessor.calculate_plot_potential(
        district, soil_type, irrigation, prev_yield
    )

    # Step 2: Crop Health Assessment
    health_score = crop_health_model.predict_health(
        ndvi, soil_moisture, temp, rainfall, days_since_sowing
    )

    # Step 3: Calculate Crop Viability Score
    viability_score = risk_engine.calculate_crop_viability_score(
        health_score, plot_potential, weather_risk, market_trend
    )

    # Step 4: Dynamic Loan Terms
    loan_terms = risk_engine.calculate_dynamic_terms(viability_score, requested_amount)

    # Step 5: Intervention Recommendations
    interventions = risk_engine.recommend_interventions(health_score, weather_risk)

    # Generate visualizations
    trend_chart = generate_health_trend_chart(health_score)

    # Format results
    assessment_summary = f"""
    ## 🌾 Krishi-Patra Assessment Report
    **Generated on:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

    ---
    ### 📊 Core Metrics
    - **Plot Potential Score:** {plot_potential:.1f}/100
    - **Current Crop Health:** {health_score:.1f}/100
    - **Crop Viability Score:** {viability_score:.1f}/100
    - **Risk Category:** {loan_terms['risk_category']}

    ---
    ### 💰 Dynamic Loan Terms
    - **Requested Amount:** ₹{requested_amount:,.0f}
    - **Approved Amount:** ₹{loan_terms['approved_amount']:,.0f}
    - **Interest Rate:** {loan_terms['interest_rate']}% p.a.
    - **Tenure:** {loan_terms['tenure_months']} months
    - **Monthly EMI:** ₹{loan_terms['emi']:,.0f}
    - **Total Repayment:** ₹{loan_terms['total_repayment']:,.0f}

    ---
    ### 🔔 Proactive Recommendations
    """

    if interventions:
        for i, rec in enumerate(interventions, 1):
            assessment_summary += f"\n**{i}. {rec['type']}:** {rec['message']}\n   *Action:* {rec['action']}\n"
    else:
        assessment_summary += "\n✅ No immediate interventions required. Crop is on track.\n"

    assessment_summary += f"""
    ---
    ### 📈 Explainability (SHAP-like Analysis)
    The viability score is influenced by:
    - Crop Health (40%): {health_score:.1f} → contributes {health_score * 0.4:.1f} points
    - Plot Potential (25%): {plot_potential:.1f} → contributes {plot_potential * 0.25:.1f} points
    - Weather Risk (20%): {100-weather_risk:.1f} → contributes {(100-weather_risk) * 0.20:.1f} points
    - Market Trend (15%): {market_trend:.1f} → contributes {market_trend * 0.15:.1f} points

    ---
    ### 🔒 Blockchain Ledger Entry
    Transaction Hash: `0x{hash(str(datetime.now()))%10**16:016x}`
    Smart Contract: Crop-Cycle-{district.replace(' ', '')}-{datetime.now().strftime('%Y%m%d')}
    """

    return assessment_summary, trend_chart

# Gradio Interface
with gr.Blocks(theme=gr.themes.Soft(), title="Krishi-Patra") as demo:
    gr.Markdown("""
    # 🌾 Krishi-Patra: Dynamic Crop Finance Platform
    ### AI-Powered Credit Assessment for Agricultural Lending
    *Transforming static loans into dynamic, data-driven financial partnerships*
    """)

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## 📍 Plot Information")
            district = gr.Textbox(label="District", value="Nashik")
            soil_type = gr.Dropdown(
                choices=["Alluvial", "Black", "Red", "Laterite", "Sandy"],
                label="Soil Type",
                value="Black"
            )
            irrigation = gr.Dropdown(
                choices=["Canal", "Borewell", "Rainfed"],
                label="Irrigation Type",
                value="Borewell"
            )
            prev_yield = gr.Slider(
                minimum=5, maximum=50, value=25,
                label="Previous Yield (Quintals/Hectare)",
                step=1
            )

            gr.Markdown("## 🛰️ Real-Time Crop Data")
            ndvi = gr.Slider(
                minimum=0.1, maximum=1.0, value=0.7,
                label="NDVI (Satellite Data)",
                step=0.05
            )
            soil_moisture = gr.Slider(
                minimum=20, maximum=80, value=55,
                label="Soil Moisture (%)",
                step=5
            )
            temp = gr.Slider(
                minimum=15, maximum=35, value=28,
                label="Temperature (°C)",
                step=1
            )
            rainfall = gr.Slider(
                minimum=0, maximum=200, value=80,
                label="Recent Rainfall (mm)",
                step=10
            )
            days_since_sowing = gr.Slider(
                minimum=0, maximum=120, value=45,
                label="Days Since Sowing",
                step=5
            )

        with gr.Column(scale=1):
            gr.Markdown("## 🌤️ External Risk Factors")
            weather_risk = gr.Slider(
                minimum=0, maximum=100, value=30,
                label="Weather Risk Score (0=Low, 100=High)",
                step=5
            )
            market_trend = gr.Slider(
                minimum=0, maximum=100, value=70,
                label="Market Price Trend (0=Poor, 100=Excellent)",
                step=5
            )

            gr.Markdown("## 💵 Loan Request")
            requested_amount = gr.Number(
                label="Requested Loan Amount (₹)",
                value=100000
            )

            assess_btn = gr.Button("🔍 Assess Loan Application", variant="primary", size="lg")

    gr.Markdown("---")

    with gr.Row():
        with gr.Column(scale=1):
            assessment_output = gr.Markdown(label="Assessment Report")
        with gr.Column(scale=1):
            chart_output = gr.Plot(label="Crop Health Trend")

    assess_btn.click(
        fn=assess_loan_application,
        inputs=[district, soil_type, irrigation, prev_yield, ndvi, soil_moisture,
                temp, rainfall, days_since_sowing, weather_risk, market_trend,
                requested_amount],
        outputs=[assessment_output, chart_output]
    )

    gr.Markdown("""
    ---
    ### 🎯 Key Features Demonstrated:
    - **Digital Twin**: Real-time crop monitoring using satellite data (NDVI)
    - **Dynamic Scoring**: Crop Viability Score that updates with conditions
    - **Proactive Interventions**: AI-driven recommendations before crisis
    - **Explainable AI**: Transparent breakdown of scoring factors
    - **Smart Contracts**: Blockchain-ready architecture for automated terms

    *Built for NABARD Hackathon | Track-1: AI-Powered Credit Assessment*
    """)

# Launch the app
if __name__ == "__main__":
    demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://dea5f62c4b91ae1507.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
